# QIIME 2 enables comprehensive end-to-end analysis of diverse microbiome data and comparative studies with publicly available data

this is a QIIME 2 Artifact CLI notebook which replicated analyses in the QIIME 2 protocol

**environment:** qiime2-2020.2

In [ ]:
pip install --upgrade c-lasso

In [ ]:
pip install c-lasso

pip install zarr

pip install plotly

In [6]:
cd ..

In [66]:
ls

ccovariates.qza			rep-seqs.qzv
classify-predictions.qza	sample-metadata-complete.tsv
classify-xtest.qza		table.qza
classify-xtraining.qza		table.qzv
classifytaxa.qza		taxonomy.qza
classifytaxa.qzv		taxonomy.qzv
features-test.qza		wcovariates.qza
filtered-table.qza		wtaxa.qza
genus_table.qza			xclr.qza
genus_table_clr.qza		xcovariates.qza
regress-predictions.qza		xtaxa.qza
regresstaxa.qza			xtest.qza
regresstaxa.qzv			xtraining.qza
rep-seqs.qza


In [67]:
cd GitHub/q2-classo/

bash: cd: GitHub/q2-classo/: No such file or directory


: 1

In [98]:
python setup.py install

pip install -e .

running install
running bdist_egg
running egg_info
writing q2_classo.egg-info/PKG-INFO
writing dependency_links to q2_classo.egg-info/dependency_links.txt
writing entry points to q2_classo.egg-info/entry_points.txt
writing top-level names to q2_classo.egg-info/top_level.txt
reading manifest file 'q2_classo.egg-info/SOURCES.txt'
writing manifest file 'q2_classo.egg-info/SOURCES.txt'
installing library code to build/bdist.macosx-10.9-x86_64/egg
running install_lib
running build_py
copying q2_classo/_func.py -> build/lib/q2_classo
copying q2_classo/plugin_setup.py -> build/lib/q2_classo
copying q2_classo/_tree.py -> build/lib/q2_classo
copying q2_classo/_summarize/_visualizer.py -> build/lib/q2_classo/_summarize
copying q2_classo/_summarize/assets/cv.html -> build/lib/q2_classo/_summarize/assets
copying q2_classo/_summarize/assets/lam-fixed.html -> build/lib/q2_classo/_summarize/assets
copying q2_classo/_summarize/assets/stabsel.html -> build/lib/q2_classo/_summarize/assets
creating build

In [122]:
qiime dev refresh-cache

QIIME is caching your current deployment for improved performance. This may take a few moments and should only happen once per deployment.


In [ ]:
cd example/Data

## Filter features

In [69]:
qiime feature-table filter-features \
  --i-table table.qza \
  --p-min-samples 20 \
  --o-filtered-table filtered-table.qza

Saved FeatureTable[Frequency] to: filtered-table.qza


## log-contrast and taxa processing

Either collapse at genus level, which is the 'easy way', but not really what we want

In [ ]:
qiime taxa collapse --i-table table.qza \
  --i-taxonomy taxonomy.qza \
  --p-level 6 \
  --o-collapsed-table genus_table.qza

qiime classo transform-features \
     --p-transformation clr \
     --p-coef 0.5 \
     --i-features genus_table.qza \
     --o-x genus_table_clr

In [70]:
 qiime classo transform-features \
     --p-transformation clr \
     --p-coef 0.5 \
     --i-features filtered-table.qza \
     --o-x xclr
     
    
qiime classo add-taxa \
	--i-features xclr.qza  \
	--i-taxa taxonomy.qza \
	--o-x xtaxa --o-aweights wtaxa

Saved FeatureTable[Design] to: xclr.qza
Saved FeatureTable[Design] to: xtaxa.qza
Saved Weights to: wtaxa.qza


## Add covariates

In [71]:
qiime classo add-covariates \
    --i-features xtaxa.qza \
    --i-weights wtaxa.qza \
    --m-covariates-file sample-metadata-complete.tsv \
    --p-to-add host_sexual_orientation host_age host_body_mass_index ethnicity \
    --p-w-to-add 0.1 1. 1. 0.1 \
    --o-new-features xcovariates \
    --o-new-c ccovariates \
    --o-new-w wcovariates

Saved FeatureTable[Design] to: xcovariates.qza
Saved ConstraintMatrix to: ccovariates.qza
Saved Weights to: wcovariates.qza


## Split table

Split data into training and testing sets : 

In [72]:
qiime sample-classifier split-table \
	--i-table xcovariates.qza \
	--m-metadata-file sample-metadata-complete.tsv \
	--m-metadata-column sCD14  \
	--p-test-size 0.2 \
	--p-random-state 42 \
	--p-stratify False \
	--o-training-table regress-xtraining \
	--o-test-table regress-xtest

Saved FeatureTable[Design] to: regress-xtraining.qza
Saved FeatureTable[Design] to: regress-xtest.qza


In [73]:
qiime sample-classifier split-table \
	--i-table xcovariates.qza \
	--m-metadata-file sample-metadata-complete.tsv \
	--m-metadata-column HIV_serostatus  \
	--p-test-size 0.2 \
	--p-random-state 42 \
	--p-stratify False \
	--o-training-table classify-xtraining \
	--o-test-table classify-xtest

Saved FeatureTable[Design] to: classify-xtraining.qza
Saved FeatureTable[Design] to: classify-xtest.qza


## Regression task 

Apply classo to the training set to solve the linear regression problem : 

In [88]:
qiime classo regress  \
    --i-features regress-xtraining.qza \
    --i-c ccovariates.qza \
    --i-weights wcovariates.qza \
    --m-y-file sample-metadata-complete.tsv \
    --m-y-column sCD14  \
    --p-concomitant \
    --p-stabsel \
    --p-cv \
    --p-path \
    --p-lamfixed \
    --p-stabsel-threshold 0.5 \
    --p-cv-seed 1 \
    --p-no-cv-one-se \
    --o-result regresstaxa

Saved CLASSOProblem to: regresstaxa.qza


## Classification task

In [126]:
qiime classo classify  \
    --i-features classify-xtraining.qza \
    --i-c ccovariates.qza \
    --i-weights wcovariates.qza \
    --m-y-file sample-metadata-complete.tsv \
    --m-y-column HIV_serostatus  \
    --p-huber \
    --p-stabsel \
    --p-cv \
    --p-path \
    --p-lamfixed \
    --p-stabsel-threshold 0.5 \
    --p-cv-seed 42 \
    --p-no-cv-one-se \
    --o-result classifytaxa

Saved CLASSOProblem to: classifytaxa.qza


## Prediction 

In [106]:
qiime classo predict \
    --i-features regress-xtest.qza \
    --i-problem regresstaxa.qza \
    --o-predictions regress-predictions.qza

Saved CLASSOProblem to: regress-predictions.qza


In [127]:
qiime classo predict \
    --i-features classify-xtest.qza \
    --i-problem classifytaxa.qza \
    --o-predictions classify-predictions.qza

Saved CLASSOProblem to: classify-predictions.qza


## Visualization

In [92]:
qiime classo summarize \
  --i-problem regresstaxa.qza \
  --i-taxa taxonomy.qza \
  --i-predictions regress-predictions.qza \
  --o-visualization regresstaxa.qzv

Saved Visualization to: regresstaxa.qzv


In [128]:
qiime classo summarize \
  --i-problem classifytaxa.qza \
  --i-taxa taxonomy.qza \
  --i-predictions classify-predictions.qza \
  --o-visualization classifytaxa.qzv \
  --verbose

Saved Visualization to: classifytaxa.qzv


In [94]:
qiime tools view regresstaxa.qzv

Press the 'q' key, Control-C, or Control-D to quit. This view may no longer be accessible or work correctly after quitting.

In [129]:
qiime tools view classifytaxa.qzv

Press the 'q' key, Control-C, or Control-D to quit. This view may no longer be accessible or work correctly after quitting.

Alternatively, one can drag&drop the file problemtaxa.qzv on : https://view.qiime2.org
Thanks to this alternative, one can also track the workflow that the qiime2 artifact did. 